In [1]:
from dotenv import load_dotenv
import os
import chromadb
import asyncio

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

> Download Ollama: https://ollama.com/download

# Extract text from pdf and store them as chunks in vector database

In [4]:
from pdfminer.high_level import extract_text
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

file = "lab_in_the_loop.pdf"
text = extract_text(file)

# convert text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64,
                                          separators=["\n\nTable","\nRow", "\n"])
chunks = splitter.split_text(text)

# create embeddings from the chunks
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
# save chunks in the vectorDB
vector_store = Chroma(collection_name="research_papers",
                      embedding_function=embeddings,
                      persist_directory="./chroma_langchain_db")
# index chunks
index = vector_store.aadd_texts(texts=chunks)


In [6]:
# see the chunks from the vector_store
all_docs = vector_store.get()
all_docs

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [ ]:

import faiss

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

# get embedding model


In [8]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Je aime le programmation.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-12T14:24:26.3421796Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6606706800, 'load_duration': 3382393100, 'prompt_eval_count': 45, 'prompt_eval_duration': 2552058300, 'eval_count': 7, 'eval_duration': 660035500, 'model_name': 'llama3.2'}, id='run--08abae3e-7188-4dfb-a575-ad7313c9d90f-0', usage_metadata={'input_tokens': 45, 'output_tokens': 7, 'total_tokens': 52})